## DATA CLEANING

En este apartado se realizaran las etapas de preprocesamiento de datos.

Trabajaremos con con nuestra tabla de hechos que se encuentra en snowflake y github.

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import KNNImputer
import ssl

In [110]:
ssl._create_default_https_context = ssl._create_unverified_context
df=pd.read_csv('https://raw.githubusercontent.com/grupohenryds03/esperanza_vida/main/datasets/EV.csv') # se importa el csv crudo

In [111]:
df.drop_duplicates(inplace = True) # eliminamos las filas duplicadas
indicadores = df['ID_INDICADOR'].unique()
indicadores.sort()
Ind_out = [16] # sacamos el indicador 12 tambien por la redundancia en indicadores
for i in indicadores:
    x = (df[df['ID_INDICADOR'] == i].VALOR.isnull().sum()/len(df[df['ID_INDICADOR'] == i])) * 100
    if x > 20:
        Ind_out.append(i)
for i in Ind_out:
    df = df[df['ID_INDICADOR'] != i] # Sacamos los indicadores dentro de Ind_Out

#-----------------------------------------------------

lista_pais=df['ID_PAIS']
lista_income=df['ID_INCOME']

#-----------------------------------------------------
label_encoder = preprocessing.LabelEncoder()
df['ID_PAIS'] = label_encoder.fit_transform(df['ID_PAIS'])
df['ID_INCOME'] = label_encoder.fit_transform(df['ID_INCOME'])

#-----------------------------------------------------
imputer = KNNImputer(n_neighbors=2, weights='distance') #Reemplazamos los Valores faltantes con KNNImputer
after = imputer.fit_transform(df) #Creamos un Data Frame usando 'after' que tiene los datos imputados.
columnas = df.columns.values
df_limpio = pd.DataFrame(after, columns=columnas) 

#-----------------------------------------------------
df_limpio.ID_PAIS=df_limpio.ID_PAIS.astype(int)
df_limpio.ID_INCOME=df_limpio.ID_INCOME.astype(int)
df_limpio.ID_CONTINENTE=df_limpio.ID_CONTINENTE.astype(int)
df_limpio.ANIO=df_limpio.ANIO.astype(int)
df_limpio.ID_INDICADOR=df_limpio.ID_INDICADOR.astype(int)
df_limpio.VALOR=round(df_limpio.VALOR,2)

In [107]:
df_limpio.to_csv('datasets/EV_limpio.csv')

creador de diccionarios para streamlit

In [ ]:
lista_X=lista_income
lista_W=list(df['ID_INCOME'])

lista_p=list(lista_pais)
lista_id=list(df_limpio.ID_PAIS)

res = {} 
for key in lista_id: 
    for value in lista_p: 
        res[key] = value 
        lista_p.remove(value) 
        break 
id=list(res)
codigo=list(res.items())
codigo_pais=list(res.values())
df_pais=pd.DataFrame(codigo, columns=['ID_PAIS','CODIGO_PAIS'])

Nation = ['United States', 'Canada','Mexico','Costa Rica','Panama','Brazil','Argentina','Chile','Uruguay','Bolivia','Peru','Egypt, Arab Rep.','Libya','South Africa','Nigeria','Morocco','Australia','China','India','Thailand','Japan','Korea, Rep.','Israel','Saudi Arabia','Malaysia','Indonesia','Russian Federation','Turkiye','Spain','Bulgaria','France','Italy','Germany','United Kingdom','Norway','Sweden','Greece']
Nation_code= ['USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA','MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','IDN','RUS','TUR','ESP','BGR','FRA','ITA','DEU','GBR','NOR','SWE','GRC']
res2 = {} 
for key in Nation_code: 
    for value in Nation: 
        res2[key] = value 
        Nation.remove(value) 
        break
print(res2)
lista_nombre=[]
for x in codigo_pais:
    lista_nombre.append(res2[x])
df_nombre=pd.DataFrame(lista_nombre,columns=['NOMBRE'])
df=pd.concat([df_pais,df_nombre],axis=1)